In [1]:
#Importing necessaray libraries

import numpy as np
import pandas as pd
import sys
sys.path.append("..")
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from numpy import asarray
from numpy import savez_compressed
from numpy import asarray
from numpy import save
from numpy import asarray
from numpy import savez_compressed
from sklearn import preprocessing

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder 
ohe = OneHotEncoder(sparse=False)

from gan import output
sys.modules["output"] = output
from gan.doppelganger import DoppelGANger
from gan.load_data import load_data
from gan.network import DoppelGANgerGenerator, Discriminator, AttrDiscriminator
from gan.output import Output, OutputType, Normalization
from gan.network import DoppelGANgerGenerator, Discriminator, \
    RNNInitialStateType, AttrDiscriminator
from gan.util import add_gen_flag, normalize_per_sample, \
    renormalize_per_sample




Bad key "text.kerning_factor" on line 4 in
C:\Users\Navid\anaconda3\envs\myenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution
C:\Users\Navid\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Navid\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Navid\anaconda3\envs\myenv\lib\site-pack

In [2]:
#creating output arrays for the features

data_feature_outputs = [
output.Output(type_=OutputType.CONTINUOUS,dim=1,normalization=Normalization.ZERO_ONE,is_gen_flag=False), 
# time intervals between transactions (Dif)
output.Output(type_=OutputType.DISCRETE,dim=20,is_gen_flag=False), 
# binarized Amount
    output.Output(type_=OutputType.DISCRETE,dim=5,is_gen_flag=False),
# Flag
output.Output(type_=OutputType.DISCRETE,dim=44,is_gen_flag=False) 
# Description    
]

In [3]:
#creating output arrays for the attributes
data_attribute_outputs = [
    output.Output(type_=OutputType.CONTINUOUS,dim=0,normalization=Normalization.MINUSONE_ONE,is_gen_flag=False)]

In [4]:
#reading 20k rows from the dataset
row_num = 20000
data=pd.read_csv('data.csv',nrows=row_num)

In [5]:
data.loc[data.loc[((data["Amount"] > -2172) & (data["Amount"] < -1671))].index, 'Amount_bin'] = 0
data.loc[data.loc[((data["Amount"] > -1671) & (data["Amount"] < -1171))].index, 'Amount_bin'] = 1
data.loc[data.loc[((data["Amount"] > -1171) & (data["Amount"] < -670))].index, 'Amount_bin'] = 2
data.loc[data.loc[((data["Amount"] > -670) & (data["Amount"] < -171))].index, 'Amount_bin'] = 3
data.loc[data.loc[((data["Amount"] > -171) & (data["Amount"] < 329))].index, 'Amount_bin'] = 4
data.loc[data.loc[((data["Amount"] > 329) & (data["Amount"] < 829))].index, 'Amount_bin'] = 5
data.loc[data.loc[((data["Amount"] > 829) & (data["Amount"] < 1329))].index, 'Amount_bin'] = 6
data.loc[data.loc[((data["Amount"] > 1329) & (data["Amount"] < 1529))].index, 'Amount_bin'] = 7
data.loc[data.loc[((data["Amount"] > 1529) & (data["Amount"] < 1729))].index, 'Amount_bin'] = 8
data.loc[data.loc[((data["Amount"] > 1729) & (data["Amount"] < 1929))].index, 'Amount_bin'] = 9
data.loc[data.loc[((data["Amount"] > 1929) & (data["Amount"] < 2130))].index, 'Amount_bin'] = 10
data.loc[data.loc[((data["Amount"] > 2130) & (data["Amount"] < 2329))].index, 'Amount_bin'] = 11
data.loc[data.loc[((data["Amount"] > 2329) & (data["Amount"] < 2529))].index, 'Amount_bin'] = 12
data.loc[data.loc[((data["Amount"] > 2529) & (data["Amount"] < 3029))].index, 'Amount_bin'] = 13
data.loc[data.loc[((data["Amount"] > 3029) & (data["Amount"] < 3529))].index, 'Amount_bin'] = 14
data.loc[data.loc[((data["Amount"] > 3529) & (data["Amount"] < 4029))].index, 'Amount_bin'] = 15
data.loc[data.loc[((data["Amount"] > 4029) & (data["Amount"] < 4529))].index, 'Amount_bin'] = 16
data.loc[data.loc[((data["Amount"] > 4529) & (data["Amount"] < 5029))].index, 'Amount_bin'] = 17
data.loc[data.loc[((data["Amount"] > 5029) & (data["Amount"] < 6746))].index, 'Amount_bin'] = 18
data['Amount'] = data['Amount'].fillna(19)
data['Amount'] = data['Amount_bin']
data = data.drop('Amount_bin', axis = 'columns')

    

In [6]:
#creating arrays for categorical & continuous features
categorical = ['Amount','Flag','Description'] 
continuous =['Balance','Dif']

#concatenating the arrays
cols = categorical + continuous

In [7]:
#number of unique categories in each categorical feature
categories_n = []

In [8]:
#filling number of distinct categories in the categories_n array
for cat in categorical:
    
    #store the number of distinct categories in each category 
    ohe.fit_transform(data[[cat]])
    ohe_Cat = ohe.categories_
    
    #function to concatenate the number into the categories_n array
    def get_all_elements_in_cat(list):
        count = 0
        for element in ohe_Cat:
            count += len(element)
        return count
    categories_n.append(get_all_elements_in_cat(ohe_Cat))


print(categories_n)

[20, 5, 44]


In [9]:
#processor function to OneHotEncode the features
def processor(user):
    
    #OneHotEncode the features 
    column_transform = make_column_transformer(
    (OneHotEncoder(), ['Flag','Description']), 
      remainder = 'passthrough')
    OHencode=column_transform.fit_transform(user)
    
    #put the values in a dataframe
    new_user = pd.DataFrame(OHencode)
    
    #rename the columns back to their original names
    col = (len(new_user.columns))-1
    col_1 = (len(new_user.columns))-2
    col_2 = (len(new_user.columns))-3
    new_user.rename(columns = {col_1:'Balance'}, inplace = True)
    new_user.rename(columns = {col:'Dif'}, inplace = True)
    new_user.rename(columns = {col_2:'Amount'}, inplace = True)
    new_user['Dif'] = new_user['Dif'].fillna(0)
    
    return new_user

In [10]:
def format_data(data, cols, nsequences=200, Lmax=100, cardinality=70):
    
    #unique account ids
    idd=list(data['Account_id'].unique()) 
    
    #format date
    data.Date = pd.to_datetime(data.Date)
    

    #appending Dif to the dataset and adding random numbers to it
    data['Dif']=np.random.randint(0,30*24*3600, size=row_num)
    
    #fill in the Numpy arrays with zeros with the needed sizes
    data_all = np.zeros((nsequences,Lmax,cardinality))
    data_attribut=np.zeros((nsequences))
    data_gen_flag=np.zeros((nsequences,Lmax))
    
    
    for i,ids in enumerate(idd[:nsequences]):
        
        #make a new dataset equivalent to 'data' with unique account ids
        user = data[data.Account_id==ids]
        user=user.sort_values(by='Date')
        
        #determine row differences and fill in the NaN values
        user['Dif']=user.Date.diff(1).iloc[1:] 
        user['Dif']=user['Dif'].dt.seconds
        user['Dif'] = user['Dif'].fillna(user['Dif'].mean())
        user['Dif'] = user['Dif'].fillna(0)
        
        #drop out the unnecessary columns
        user = user[cols]
        
        
        # copy dataset to preprocess
        processed_data = processor(user)

        #processed_data.to_csv('raw_data.csv', index=False)
         
        print(processed_data)
        #fill Data_attribut with the values of normalized 'Balance
        data_attribut[i] = processed_data['Balance'].values[0]
        
        
        #fill data_gen_flag with zeros & ones 
        data_gen_flag[i,:len(user)]=1
        
        processed_array = np.asarray(processed_data.iloc[:])
        #print(processed_data.shape[1])
        user_col = processed_data.shape[1]
        data_all[i,:len(user),:user_col]=processed_array

    return data_all, data_attribut, data_gen_flag

In [11]:
#calling the function to produce the input
data_all, data_attribut, data_gen_flag = format_data(data,cols)

     0    1    2    3    4  Amount  Balance           Dif
0  0.0  1.0  0.0  0.0  1.0     4.0   349.00  42819.333333
1  1.0  0.0  0.0  1.0  0.0     4.0   280.52  48943.000000
2  1.0  0.0  1.0  0.0  0.0     4.0   247.38  78655.000000
3  1.0  0.0  0.0  1.0  0.0     4.0   230.17    860.000000
     0    1    2    3    4  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  1.0     4.0  1012.75  19800.0
1  1.0  0.0  1.0  0.0  0.0     4.0  1003.76  36000.0
2  0.0  1.0  0.0  1.0  0.0     4.0   982.76   3600.0
     0    1    2    3    4    5    6    7  Amount  Balance       Dif
0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0     4.0  2693.00  32733.75
1  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0     0.0   943.00  25200.00
2  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0   921.00      0.00
3  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0   913.01  21600.00
4  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0   797.38  84135.00
     0    1    2    3    4    5    6    7    8    9   10   11  Amount  \
0  0.0  0.0

2  1.0  0.0  1.0  0.0  0.0     3.0  -621.43      0.0
     0    1    2    3    4    5    6    7    8    9   10  Amount  Balance  \
0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0   260.96   
1  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0   250.96   
2  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     4.0   226.29   
3  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0   190.29   
4  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     4.0   158.29   
5  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     4.0    99.34   
6  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0    14.0  3174.59   

            Dif  
0  34509.666667  
1   3600.000000  
2  28800.000000  
3  61200.000000  
4  39600.000000  
5  68400.000000  
6   5458.000000  
     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  1.0  0.0  0.0  0.0     2.0   545.00  29991.5
1  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0   512.17  32400

     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  0.0  1.0  0.0     4.0   -36.00  27900.0
1  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0   -59.00  82800.0
2  0.0  1.0  1.0  0.0  0.0  0.0  0.0     4.0  -100.82      0.0
3  0.0  1.0  0.0  0.0  1.0  0.0  0.0     2.0 -1210.82  10800.0
4  1.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0 -1230.82  18000.0
     0    1    2    3    4    5    6    7  Amount  Balance      Dif
0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0     4.0   -22.00  28800.0
1  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0   -29.99   7200.0
2  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0     4.0   -74.80  75600.0
3  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0   -74.80  21600.0
4  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0  -111.47  10800.0
     0    1    2    3    4    5    6    7  Amount  Balance           Dif
0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0     4.0   457.17  35754.666667
1  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0   434.17  82800.00

     0    1    2    3    4    5  Amount  Balance      Dif
0  0.0  0.0  1.0  1.0  0.0  0.0     4.0   997.01  14983.0
1  0.0  1.0  0.0  0.0  1.0  0.0    14.0  4109.21  20801.0
2  1.0  0.0  0.0  0.0  0.0  1.0     4.0  4080.53   9165.0
     0    1    2    3    4    5    6    7    8    9  Amount  Balance      Dif
0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0     4.0   394.33  35388.5
1  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     4.0   386.34   7200.0
2  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0   361.34  28800.0
3  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0   312.27  68400.0
4  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0    10.0  2439.50  65966.0
5  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     4.0  2434.24  41875.0
6  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     4.0  2428.25     90.0
     0    1    2    3    4    5  Amount  Balance      Dif
0  1.0  0.0  1.0  0.0  0.0  0.0     4.0   655.01  38400.0
1  0.0  1.0  0.0  0.0  0.0  

5  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     9.0  1342.55  58389.0
     0    1    2    3    4    5    6    7  Amount  Balance       Dif
0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0     4.0  2967.05  51865.75
1  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0     4.0  2935.05  28800.00
2  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0  2852.12  83966.00
3  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0  2837.12  63633.00
4  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0  2774.62  31064.00
     0    1    2    3    4    5    6    7  Amount  Balance       Dif
0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0   320.00  30599.75
1  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0     4.0   298.00  18000.00
2  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0     4.0   257.85  18000.00
3  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     5.0   794.98  73166.00
4  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0   789.98  13233.00
     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  0.0  0.0  1.0

     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  0.0  1.0  0.0     4.0   107.00  28800.0
1  0.0  1.0  0.0  0.0  1.0  0.0  0.0     4.0    71.00  43200.0
2  0.0  1.0  0.0  1.0  0.0  0.0  0.0     1.0 -1186.00  50400.0
3  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0 -1215.83   3600.0
4  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0 -1220.32  18000.0
     0    1    2    3    4  Amount  Balance      Dif
0  1.0  0.0  1.0  0.0  0.0     4.0   313.00  64800.0
1  1.0  1.0  0.0  0.0  0.0     2.0  -627.00  79200.0
2  1.0  0.0  0.0  0.0  1.0     4.0  -667.58  36000.0
3  1.0  0.0  0.0  1.0  0.0     4.0  -690.58  79200.0
     0    1    2    3    4    5    6  Amount  Balance           Dif
0  0.0  0.0  1.0  0.0  0.0  0.0  1.0     4.0   974.00  32399.666667
1  1.0  0.0  0.0  1.0  0.0  0.0  0.0     3.0   755.51  26366.000000
2  1.0  0.0  0.0  0.0  1.0  0.0  0.0     4.0   728.17  17400.000000
3  0.0  1.0  0.0  0.0  0.0  1.0  0.0     4.0   723.17  53433.000000
     0    1    2 

     0    1    2    3    4    5  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  1.0  0.0     4.0   207.17  34800.0
1  1.0  0.0  1.0  0.0  0.0  0.0     4.0   199.18  79200.0
2  0.0  1.0  0.0  0.0  0.0  1.0     4.0   169.19  18000.0
3  0.0  1.0  0.0  1.0  0.0  0.0     4.0   118.46   7200.0
     0    1    2    3    4    5    6    7  Amount  Balance      Dif
0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0     4.0   647.35   6300.0
1  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0   637.40  18000.0
2  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0     0.0 -1097.60      0.0
3  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0    15.0  2503.32    569.0
4  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0  2481.32   6631.0
     0    1    2    3    4  Amount  Balance      Dif
0  1.0  0.0  1.0  0.0  0.0     4.0   112.05  41400.0
1  0.0  1.0  0.0  0.0  1.0     4.0    77.80  14400.0
2  0.0  1.0  0.0  1.0  0.0     2.0  -720.20  68400.0
     0    1    2    3    4    5  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  1.0  0.0 

     0    1    2    3    4    5    6    7    8    9  Amount  Balance  \
0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0    14.0  3517.33   
1  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     4.0  3468.62   
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     4.0  3390.04   
3  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     4.0  3382.88   
4  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     1.0  2207.88   
5  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     4.0  2184.88   
6  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0  2172.14   

            Dif  
0  19480.833333  
1   3417.000000  
2  19166.000000  
3  20057.000000  
4  36376.000000  
5      0.000000  
6  37869.000000  
     0    1    2    3    4    5    6  Amount  Balance      Dif
0  1.0  0.0  0.0  1.0  0.0  0.0  0.0    15.0  4015.92  58876.0
1  0.0  0.0  1.0  0.0  0.0  0.0  1.0     4.0  3994.92  72228.0
2  0.0  0.0  1.0  0.0  0.0  1.0  0.0     1.0  2474.92  43200.0
3  0.0  1.0  0.0  0.0  1.0 

In [12]:
#setting some variables
sample_len = 100
epoch = 100
batch_size = 20
d_rounds = 2
g_rounds = 1
d_gp_coe = 10.0
attr_d_gp_coe = 10.0
g_attr_d_coe = 1.0

In [13]:
# normalize data
(data_feature, data_attribute, data_attribute_outputs,
 real_attribute_mask) = normalize_per_sample(
        data_all, data_attribut, data_feature_outputs,
        data_attribute_outputs)

# add generation flag to features
data_feature, data_feature_outputs = add_gen_flag(
    data_feature, data_gen_flag, data_feature_outputs, sample_len)
generator = DoppelGANgerGenerator(
    feed_back=False,
    noise=True,
    feature_outputs=data_feature_outputs,
    attribute_outputs=data_attribute_outputs,
    real_attribute_mask=real_attribute_mask,
    sample_len=sample_len,
    feature_num_units=100,
    feature_num_layers=2)
#calling the discriminator functions
discriminator = Discriminator()
attr_discriminator = AttrDiscriminator()

In [14]:
#creating folders to store the training results
checkpoint_dir = r'C:\Users\Navid\Videos\DöppleGANger\test\checkpoint'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
sample_dir = r'C:\Users\Navid\Videos\DöppleGANger\test\sample'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
time_path = r'C:\Users\Navid\Videos\DöppleGANger\test\time.txt'

#set some values
epoch = 100
batch_size = 50
g_lr = 0.0001
d_lr = 0.0001
vis_freq = 50
vis_num_sample = 5
d_rounds = 2
g_rounds = 1
d_gp_coe = 10.0
attr_d_gp_coe = 10.0
g_attr_d_coe = 1.0
extra_checkpoint_freq = 30
num_packing = 1


In [15]:
#start the build() and train() functions
run_config = tf.ConfigProto()
with tf.Session(config=run_config) as sess:
    gan = DoppelGANger(
        sess=sess, 
        checkpoint_dir=checkpoint_dir,
        sample_dir=sample_dir,
        time_path=time_path,
        epoch=epoch,
        batch_size=batch_size,
        data_feature=data_feature,
        data_attribute=data_attribute,
        real_attribute_mask=real_attribute_mask,
        data_gen_flag=data_gen_flag,
        sample_len=sample_len,
        data_feature_outputs=data_feature_outputs,
        data_attribute_outputs=data_attribute_outputs,
        vis_freq=vis_freq,
        vis_num_sample=vis_num_sample,
        generator=generator,
        discriminator=discriminator,
        attr_discriminator=attr_discriminator,
        d_gp_coe=d_gp_coe,
        attr_d_gp_coe=attr_d_gp_coe,
        g_attr_d_coe=g_attr_d_coe,
        d_rounds=d_rounds,
        g_rounds=g_rounds,
        g_lr=g_lr,
        d_lr=d_lr,
        num_packing=num_packing,
        extra_checkpoint_freq=extra_checkpoint_freq)
    gan.build()
    gan.train()
    print('Training completed!')

Layers of DoppelGANgerGenerator
[<tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/linear/dense/kernel:0' shape=(5, 100) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/linear/dense/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/beta:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/gamma:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/moving_mean:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/moving_variance:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer1/linear/dense/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer1/linear/dense/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [24:09<00:00, 14.50s/it]

Training completed!
